In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from keras.optimizers import Adam

In [2]:
import pandas as pd
import optuna
import yfinance as yf
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas_ta as ta
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Nadam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from keras.utils import plot_model
from keras.layers import Dropout, Dense, GRU, LSTM, Flatten
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore', category = pd.errors.PerformanceWarning)

In [4]:
# Pobieramy dane dotyczące BTC z yahoo
BTC = yf.download('BTC-USD', start = '2020-01-01', end = '2022-02-23')

BTC['Target'] = (BTC['Close'].diff() > 0).astype(int)

def add_lagged_features(data, column_name, n_days):
    for i in range(1, n_days + 1):
        lagged_column_name = f'{column_name}_{i}d_back'
        data[lagged_column_name] = data[column_name].shift(i)
    return data

BTC = add_lagged_features(BTC, 'Close', 30)
BTC = add_lagged_features(BTC, 'Volume', 30)
BTC = add_lagged_features(BTC, 'High', 1)
BTC = add_lagged_features(BTC, 'Low', 1)

# df = pd.DataFrame()
# df = BTC.copy()
# print(BTC.head())
print(len(BTC))
print(BTC.shape)

[*********************100%%**********************]  1 of 1 completed

784
(784, 69)


In [5]:
# SMA
BTC['SMA'] = ta.sma(BTC['Close_1d_back'], length = 30)

# MFI 
BTC['MFI'] = ta.mfi(BTC['High_1d_back'], BTC['Low_1d_back'], 
                    BTC['Close_1d_back'], BTC['Volume_1d_back'], length = 29)

# RSI 
BTC['RSI'] = ta.rsi(BTC['Close_1d_back'], length = 29)

# Bollinger Bands
bollinger = ta.bbands(BTC['Close_1d_back'], length = 30, std = 2)
BTC['BB_Upper'] = bollinger['BBU_30_2.0']
BTC['BB_Middle'] = bollinger['BBM_30_2.0']
BTC['BB_Lower'] = bollinger['BBL_30_2.0']

# CCI 
BTC['CCI'] = ta.cci(BTC['High_1d_back'], BTC['Low_1d_back'], 
                    BTC['Close_1d_back'], length = 30)

In [6]:
high_series = BTC['High_1d_back']
low_series = BTC['Low_1d_back']
close_series = BTC['Close_1d_back']

# Wprowadzenie jasnych parametrów dla kanału Donchian
donchian = ta.donchian(high = high_series, low = low_series, close = close_series, lower_length = 30, upper_length = 30)

# Przypisanie wyników kanału Donchian do DataFrame
BTC['Donchian_High'] = donchian['DCU_30_30']
BTC['Donchian_Low'] = donchian['DCL_30_30']

In [7]:
BTC = BTC.drop('Adj Close', axis=1)
BTC = BTC.dropna().reset_index(drop = True)
# BTC

In [8]:
dataset = pd.DataFrame()
dataset = BTC[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA', 'MFI', 'RSI', 'BB_Upper', 
          'BB_Middle', 'BB_Lower', 'CCI', 'Donchian_High', 'Donchian_Low', 'Target']].copy()
# dataset

In [9]:
n_steps_in = 20
output_features = 2

In [10]:
X_value = pd.DataFrame(dataset.iloc[:, :])
X_value = X_value.drop('Target', axis='columns')
# X_value

In [11]:
y_value = pd.DataFrame(dataset.Target)
y_value = to_categorical(y_value)
# y_value

In [12]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(X_value)
X_value = scaler.fit_transform(X_value)

In [13]:
def get_X_y(X_data, y_data):
    X = list()
    y = list()

    length = len(X_data)
    for i in range(0, length-n_steps_in, 1):
        X_value = X_data[i: i + n_steps_in][:, :]
        # y_value = y_data[i + n_steps_in: i + (n_steps_in + n_steps_out)][:, :]
        y_value = y_data[i + n_steps_in]
        if len(X_value) == n_steps_in and len(y_value) == output_features:
            X.append(X_value)
            y.append(y_value)

    return np.array(X), np.array(y)

In [14]:
def split_train_test(data):
    train_size = int(0.8 * len(X))
    data_train = data[0:train_size]
    data_test = data[train_size:]
    return data_train, data_test

In [15]:
X, y = get_X_y(X_value, y_value)
print('X shape: ', X.shape)
print('y shape: ', y.shape)
train_features, test_features = split_train_test(X) 
train_target, test_target = split_train_test(y) 

print('Train features shape: ', train_features.shape)
print('Test features shape: ', test_features.shape)
print('Train features shape: ', train_target.shape)
print('Test features shape: ', test_target.shape)

X shape:  (734, 20, 14)
y shape:  (734, 2)
Train features shape:  (587, 20, 14)
Test features shape:  (147, 20, 14)
Train features shape:  (587, 2)
Test features shape:  (147, 2)


In [16]:
input_dim = train_features.shape[1]
input_feature_size = train_features.shape[2]
output_dim = train_target.shape[1]

In [17]:
N_EPOCH = 50
BATCH_SIZE = 64

def basic_simpleRNN(input_dim, output_dim, input_feature_size):
    model = Sequential()
    model.add(SimpleRNN(units=63, return_sequences=True, activation='relu', input_shape=(input_dim, input_feature_size)))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(250, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_features, train_target, epochs = N_EPOCH, 
                        validation_data = (test_features, test_target),
                        batch_size = BATCH_SIZE, verbose = 2, shuffle = False)

    return model, history

In [18]:
model, history = basic_simpleRNN(input_dim, output_dim, input_feature_size)

C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
10/10 - 18s - 2s/step - accuracy: 0.5366 - loss: 0.9192 - val_accuracy: 0.4966 - val_loss: 0.8144
Epoch 2/50
10/10 - 1s - 81ms/step - accuracy: 0.4974 - loss: 0.7797 - val_accuracy: 0.5238 - val_loss: 0.6981
Epoch 3/50
10/10 - 1s - 134ms/step - accuracy: 0.4974 - loss: 0.7724 - val_accuracy: 0.4694 - val_loss: 0.6995
Epoch 4/50
10/10 - 1s - 117ms/step - accuracy: 0.5571 - loss: 0.7098 - val_accuracy: 0.5306 - val_loss: 0.6935
Epoch 5/50
10/10 - 1s - 103ms/step - accuracy: 0.5349 - loss: 0.7063 - val_accuracy: 0.4966 - val_loss: 0.7203
Epoch 6/50
10/10 - 1s - 97ms/step - accuracy: 0.5537 - loss: 0.7123 - val_accuracy: 0.4898 - val_loss: 0.7073
Epoch 7/50
10/10 - 1s - 124ms/step - accuracy: 0.5554 - loss: 0.6991 - val_accuracy: 0.4762 - val_loss: 0.7085
Epoch 8/50
10/10 - 1s - 100ms/step - accuracy: 0.5315 - loss: 0.7192 - val_accuracy: 0.4830 - val_loss: 0.7368
Epoch 9/50
10/10 - 1s - 136ms/step - accuracy: 0.5758 - loss: 0.6899 - val_accuracy: 0.4762 - val_loss: 0.7370
Epoch

In [19]:
# Ocena modelu na danych treningowych
train_performance = model.evaluate(train_features, train_target)
print('Train loss:', train_performance[0])
print('Train accuracy:', train_performance[1])

test_performance = model.evaluate(test_features, test_target)
print('Test loss:', test_performance[0])
print('Test accuracy:', test_performance[1])

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6271 - loss: 0.6195
Train loss: 0.528897762298584
Train accuracy: 0.7035775184631348
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4708 - loss: 1.3242
Test loss: 1.0857549905776978
Test accuracy: 0.4761904776096344


In [20]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step
272
141
129
45
Accuracy (ACC): 0.7035775127768313
True Positive Rate (TPR): 0.8580441640378549
False Positive Rate (FPR): 0.4777777777777778
True Negative Rate (TNR): 0.5222222222222223
Positive Predictive Value (PPV): 0.6783042394014963
Negative Predictive Value (NPV): 0.7580645161290323


In [21]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
24
46
27
50
Accuracy (ACC): 0.47619047619047616
True Positive Rate (TPR): 0.32432432432432434
False Positive Rate (FPR): 0.3698630136986301
True Negative Rate (TNR): 0.6301369863013698
Positive Predictive Value (PPV): 0.47058823529411764
Negative Predictive Value (NPV): 0.4791666666666667
